In [1]:
import numpy as np
import universities
import csv

In [2]:
def import_data(file_name):
    data = []
    header = []

    count = 0
    done = False
    with open(file_name, 'rb') as sql_file:
        for line in sql_file:
            count += 1
            if line.startswith(b'INSERT'):
                line = str(line, "ISO-8859-1")[33:].split("\'")
                new_line = []
                for i in range(len(line)):
                    if '),(' in line[i]:
                        data.append(new_line)
                        new_line = []
                        new_line.append(line[i].replace('),(', ''))
                    else:
                        new_line.append(line[i])

            elif count >= 25:
                if line.startswith(b'  PRIMARY'):
                    done = True
                    continue
                elif done:
                    continue
                else:
                    line = str(line, "ISO-8859-1")
                    string = ''
                    start = False
                    for i in range(len(line)):
                        if line[i] == '"' and not start:
                            start = True
                            continue
                        elif line[i] == '"' and start:
                            break
                        if start:
                            string += line[i]
                    header.append(string)

    return data, header

In [3]:
people, people_header = import_data('./data/crunchbase_data/cb_people.sql')
degrees, degrees_header = import_data('./data/crunchbase_data/cb_degrees.sql')
companies, companies_header = import_data('./data/crunchbase_data/cb_objects.sql')
relationships, relationships_header = import_data('./data/crunchbase_data/cb_relationships.sql')
milestones, milestones_header = import_data('./data/crunchbase_data/cb_milestones.sql')
funding_rounds, funding_rounds_header = import_data('./data/crunchbase_data/cb_funding_rounds.sql')

In [5]:
unique_companies = []
for i in range(len(people)):
    if people[i][-1] not in unique_companies:
        unique_companies.append(people[i][-1])

In [113]:
len(people)

226696

In [194]:
companies_dict = {}
for i in range(len(companies)):
    for j in range(len(unique_companies)):
        if unique_companies[j] in companies[i]:
            companies_dict[unique_companies[j]] = companies[i]
            break

In [8]:
company_by_id = {}
for i in range(len(unique_companies)):
    try:
        company_by_id[companies_dict[unique_companies[i]][1]] = unique_companies[i]
    except:
        pass

In [32]:
len(company_by_id)

2519

In [89]:
venture_round_funding_by_company = {}
for i in range(len(funding_rounds)):
    if funding_rounds[i][5] == 'venture':
        try:
            venture_round_funding_by_company[company_by_id[funding_rounds[i][1]]] = int(funding_rounds[i][8].split(',')[1])
        except:
            pass

In [90]:
rounds = {}
for i in range(len(funding_rounds)):
    if funding_rounds[i][5] not in rounds:
        rounds[funding_rounds[i][5]] = 1
    else:
        rounds[funding_rounds[i][5]] += 1

In [91]:
rounds

{'a': 37,
 'angel': 13076,
 'b': 2,
 'c': 3,
 'crowdfunding': 111,
 'debt_round': 5,
 'grant': 1,
 'other': 4194,
 'post-ipo': 87,
 'private-equity': 1043,
 'seed': 91,
 'series-a': 9827,
 'series-b': 4888,
 'series-c+': 4213,
 'unattributed': 42,
 'venture': 15293}

In [193]:
founders = {}
for i in range(len(people)):
    full_name = people[i][3] + ' ' + people[i][5]
    if full_name in founders and founders[full_name]['id'] == people[i][1]:
        continue
    else:
        founders[full_name] = {}
        founders[full_name]['id'] = people[i][1]
        founders[full_name]['company'] = people[i][-1]
        try:
            founders[full_name]['company_id'] = companies_dict[people[i][-1]][1]
        except:
            pass
        
        try:
            founders[full_name]['venture_round'] = angel_round_funding_by_company[people[i][-1]]
        except:
            pass

        for k in range(len(degrees)):
            if degrees[k][1] == people[i][1]:
                founders[full_name]['degree'] = degrees[k][3]
                founders[full_name]['majors'] = degrees[k][5]
                founders[full_name]['school'] = degrees[k][7]
                break

In [100]:
company_founders = {}
for founder in founders:
    if founders[founder]['company'] not in unique_companies:
        continue
    else:
        if founders[founder]['company'] not in company_founders:
            company_founders[founders[founder]['company']] = {}
            company_founders[founders[founder]['company']][founder] = founders[founder]
        elif founders[founder]['company'] in company_founders:
            company_founders[founders[founder]['company']][founder] = founders[founder]

In [101]:
company_founders['Facebook']

{'Adam D\\': {'company': 'Facebook',
  'degree': 'BS',
  'id': 'p:1531',
  'majors': 'Computer Science',
  'school': 'California Institute of Technology'},
 'Andrew Bosworth': {'company': 'Facebook',
  'degree': 'BA',
  'id': 'p:52929',
  'majors': 'Computer Science',
  'school': 'Harvard University'},
 'Ari Steinberg': {'company': 'Facebook',
  'degree': 'MS',
  'id': 'p:58157',
  'majors': 'Computer Science',
  'school': 'Stanford University'},
 'Beau Hartshorne': {'company': 'Facebook', 'id': 'p:54626'},
 'Benjamin Joe': {'company': 'Facebook',
  'degree': 'MBA',
  'id': 'p:64734',
  'majors': 'Business',
  'school': 'Harvard Business School'},
 'Carolyn Everson': {'company': 'Facebook',
  'degree': 'MBA',
  'id': 'p:91903',
  'majors': 'Business Administration',
  'school': 'Harvard Business School'},
 'Chris Cox': {'company': 'Facebook',
  'degree': 'BS',
  'id': 'p:36151',
  'majors': 'Symbolic Systems',
  'school': 'Stanford University'},
 'Christian Hernandez': {'company': 'Fac

In [102]:
unique_degrees = {}
for founder in founders:
    try:
        degree = founders[founder]['degree']
        if degree in unique_degrees:
            unique_degrees[degree] += 1
        else:
            unique_degrees[degree] = 1
    except:
        continue

In [103]:
sorted(unique_degrees.items(), key=lambda x: x[1], reverse=True)

[('BS', 5008),
 ('BA', 3325),
 ('MBA', 3065),
 ('MS', 1870),
 ('', 1833),
 ('PhD', 739),
 ('BBA', 290),
 ('JD', 283),
 ('Degree', 248),
 ('MA', 245),
 ('BE', 234),
 ('BTECH', 110),
 ('BFA', 86),
 ('BCOM', 70),
 ('MD', 68),
 ('Diploma', 57),
 ('MENG', 44),
 ('BENG', 41),
 ('LLB', 38),
 ('MSE', 36),
 ('Engineering', 29),
 ('Engineer', 27),
 ('AS', 26),
 ('BASC', 21),
 ('Graduated', 21),
 ('AA', 21),
 ('Bachelor of Arts (B.A.)', 19),
 ('MPHIL', 19),
 ('Master Degree', 18),
 ('Graduation', 18),
 ('Bachelor\x92s Degree', 17),
 ('Computer Science', 16),
 ('BSME', 14),
 ('Bachelor of Fine Arts', 14),
 ('LLM', 13),
 ('Certificate', 13),
 ('Bachelor of Science (B.S.)', 13),
 ('BCS', 12),
 ('SB', 11),
 ('Bachelor of Arts (BA)', 11),
 ('Master of Business Administratio', 11),
 ('Bachelor of Science (BS)', 11),
 ('BBS', 10),
 ('ME', 10),
 ('degree', 10),
 ('Bachelor\\', 10),
 ('Business', 10),
 ('Bachelorâ\x80\x99s degree', 10),
 ('Graduate', 9),
 ('Business Administration', 9),
 ('MPA', 9),
 ('Ba

In [115]:
top_degrees = ['BA', 'BS', 'MBA', 'MS', 'PhD', 'BBA', 'JD', 'MA', 'BE']

In [116]:
unique_majors = {}
for founder in founders:
    try:
        majors = founders[founder]['majors']
        if majors in unique_majors:
            unique_majors[majors] += 1
        else:
            unique_majors[majors] = 1
    except:
        continue

In [117]:
top_majors = [x[0] for x in list(sorted(unique_majors.items(), key=lambda x: x[1], reverse=True)[1:50]) \
     if x[0] not in top_degrees and x[0] != 'Degree']

In [165]:
baseline_founders = {}
for founder in founders:
    try:
        degree = founders[founder]['degree']
        if degree in top_degrees:
            majors = founders[founder]['majors']
            if majors in top_majors:
                baseline_founders[founder] = founders[founder]
    except:
        pass

In [166]:
len(baseline_founders)

5527

In [167]:
train_data = []
for founder in baseline_founders:
    line = []
    for key in baseline_founders[founder]:
        line.append(baseline_founders[founder][key])
    train_data.append(line)

In [187]:
train_universities = []
for i in range(len(train_data)):
    if train_data[i][0].startswith('p:') and len(train_data[i]) != 6:
        if train_data[i][-1] not in train_universities:
            train_universities.append(train_data[i][-1])
    elif train_data[i][0].startswith('p:') and len(train_data[i]) == 6 and train_data[i][2].startswith('c:'):
        if train_data[i][-2] not in train_universities:
            train_universities.append(train_data[i][-1])
    else:
        if train_data[i][-2] not in train_universities:
            train_universities.append(train_data[i][-2])

In [188]:
train_universities

['Stanford University',
 'University of Chicago, School of Law',
 'University of Illinois at Urbana-Champaign',
 'University of Michigan',
 'Tulane University',
 'The University of Calgary',
 '1979-01-01',
 '2013-07-23 16:45:53',
 'Boston College',
 'State University of New York at Buffalo',
 'University of California, Berkeley',
 '1993-01-01',
 'Fordham University',
 'San Jose State University',
 'New York University (NYU)',
 'Stanford University, Law School',
 'St. Norbert College',
 '2000-01-01',
 '1978-01-01',
 'Yale University',
 'West Point',
 '1988-01-01',
 '1986-01-01',
 'Carleton University',
 'Bucknell',
 'Rutgers, The State University of New Jersey-New Brunswick',
 '2008-04-10 21:43:48',
 'University of Central Florida',
 'University of Washington',
 'Brown University',
 '1981-01-01',
 'Fairfiled University',
 'Stanford University',
 'University of Montreal',
 '2010-08-19 08:41:17',
 '2003-01-01',
 'Columbia University',
 '1992-01-01',
 '1992-01-01',
 '2013-07-24 11:47:02',
